In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt 
import cmocean 
plt.rcParams.update({'font.size': 16})


# function to generate monthly datetime list
def generate_monthly_datetimes(start, end):
    dates = []
    current = start
    while current <= end:
        dates.append(current)
        year = current.year + (current.month // 12)
        month = current.month % 12 + 1
        current = datetime(year, month, 1)
    return dates

# Generate datetime array
t_start = datetime(2015, 1, 1)
t_end = datetime(2025, 5, 1)
t = generate_monthly_datetimes(t_start, t_end)
n = len(t)

# key dates
t_2019_raise = dt.datetime(2019, 7, 1)
t_2021_raise = dt.datetime(2021, 7, 1)
t_2022_raise = dt.datetime(2022, 7, 1)
t_apr23_raise = dt.datetime(2023, 4, 1)
t_2022contract_ratification = dt.datetime(2022,12,24)
t_grievance_submission = dt.datetime(2023,7,1)
t_oct23_raise = dt.datetime(2023, 10, 1)
t_oct24_raise = dt.datetime(2024, 10, 1)
t_2023students_start = dt.datetime(2023,9,1)
t_2024students_start = dt.datetime(2024,9,1)
t_50settlement_start = dt.datetime(2024,4,1)
t_50settlement_end = dt.datetime(2024,7,1)
t_stepgrievance_ruling = dt.datetime(2025,7,7)


# salary arrays
n = len(t)
gsr_salary_old_student_prequal = np.zeros(n)
gsr_salary_old_student_postqual = np.zeros(n)
gsr_salary_2023student = np.zeros(n)*np.nan 
gsr_salary_2024student = np.zeros(n)*np.nan

## monthly 

step3_fte_2022 = 5909.58
step4_fte_2022 = 6367.58
step6_fte_2022 = 7392.83

step3_fte_2023 = 6287.83
step4_fte_2023 = 6775.17
step6_fte_2023 = 7866.00

step3_fte_2024 = 6688.33
step4_fte_2024 = 7206.75
step6_fte_2024 = 8367.17


# theoretical money we should have been making SINCE 2022 CONTRACT 
c_start = t_apr23_raise
c_end = datetime(2025, 7, 12)
contract_dt = np.array(generate_monthly_datetimes(c_start, c_end))
clen = len(contract_dt)

gsr_contract_salary = np.zeros(clen)*np.nan
gsr_contract_salary[contract_dt>=t_apr23_raise] = step6_fte_2023*0.5 
gsr_contract_salary[contract_dt>t_oct24_raise] = step6_fte_2024*0.5 

cumulative_contracted_wages = np.sum(gsr_contract_salary)

# how much money did we get? 
gsr_prequal_salary = np.zeros(clen)*np.nan
gsr_prequal_salary[contract_dt>=t_apr23_raise] = step6_fte_2023*0.4
gsr_prequal_salary[contract_dt>t_settlement_end] = step6_fte_2023*0.5 
gsr_prequal_salary[contract_dt>t_oct24_raise] = step6_fte_2024*0.5

gsr_postqual_salary = np.zeros(clen)*np.nan
gsr_postqual_salary[contract_dt>=t_apr23_raise] = step6_fte_2023*0.43 
gsr_postqual_salary[contract_dt>t_settlement_end] = step6_fte_2023*0.5 
gsr_postqual_salary[contract_dt>t_oct24_raise] = step6_fte_2024*0.5 

# rough losses due to 50% 

cumulative_prequal_wages = np.sum(gsr_prequal_salary)
cumulative_postqual_wages = np.sum(gsr_postqual_salary)

prequal_losses = cumulative_prequal_wages - cumulative_contracted_wages
postqual_losses = cumulative_postqual_wages - cumulative_contracted_wages

# then for 2023/2024 cohorts due to step lowering 

gsr_2023cohort_salary = np.zeros(clen)*np.nan
gsr_2023cohort_salary[contract_dt>=t_2023students_start] = step3_fte_2023*0.5 
gsr_2023cohort_salary[contract_dt>t_oct24_raise] = step4_fte_2024*0.5 

gsr_contract_2023salary = np.zeros(clen)*np.nan
gsr_contract_2023salary[contract_dt>=t_2023students_start] = step6_fte_2023*0.5 
gsr_contract_2023salary[contract_dt>t_oct24_raise] = step6_fte_2024*0.5 

cumulative_contracted_2023_wages = np.nansum(gsr_contract_2023salary) 
cumulative_2023_wages = np.nansum(gsr_2023cohort_salary) 
losses_2023 = cumulative_2023_wages - cumulative_contracted_2023_wages

gsr_2024cohort_salary = np.zeros(clen)*np.nan
gsr_2024cohort_salary[contract_dt>=t_2024students_start] = step3_fte_2023*0.5 
gsr_2024cohort_salary[contract_dt>t_oct24_raise] = step3_fte_2024*0.5 

gsr_contract_2024salary = np.zeros(clen)*np.nan
gsr_contract_2024salary[contract_dt>=t_2024students_start] = step6_fte_2023*0.5 
gsr_contract_2024salary[contract_dt>t_oct24_raise] = step6_fte_2024*0.5 

cumulative_contracted_2024_wages = np.nansum(gsr_contract_2024salary) 
cumulative_2024_wages = np.nansum(gsr_2024cohort_salary) 
losses_2024 = cumulative_2024_wages - cumulative_contracted_2024_wages

thiscmap = cmocean.cm.thermal # remember, can do reverse with _r at end of cmap name 
cpos = np.linspace(0.1,0.85,4)
colors = [thiscmap(c) for c in cpos] 

# plot salaries over time 

fig = plt.figure(figsize=(16,8))
plt.plot(contract_dt, gsr_contract_salary, label='<<Agreed>> Wages: 50% Step 6', lw=7, ls='-.',color='red')
plt.plot(contract_dt, gsr_postqual_salary, label='Old Post-Qual: 43% Step 6',color=colors[0], lw=5)
plt.plot(contract_dt, gsr_prequal_salary, label='Old Pre-Qual: 40% Step 6',color=colors[1], lw=5)
plt.plot(contract_dt, gsr_2023cohort_salary, label='2023 Cohort: 50% Step 3-4',color=colors[2], lw=5)
plt.plot(contract_dt, gsr_2024cohort_salary, label='2024 Cohort: 50% Step 3',color=colors[3], lw=5)
plt.annotate(xy=[t_2022contract_ratification,3400],text='Older Students \n  Post-Qualifying \n  (43% FTE)',color=colors[0])
plt.annotate(xy=[t_2022contract_ratification,3150],text='Older Students \n  Pre-Qualifying \n  (40% FTE)',color=colors[1])
plt.annotate(xy=[t_2023students_start,3120],text='2023 Cohort \n ',color=colors[2],va='bottom')
plt.annotate(xy=[t_2024students_start,3120],text='2024 Cohort \n ',color=colors[3],va='bottom',ha='center')

# add key dates 
plt.vlines(x=[t_2022contract_ratification,t_grievance_submission,t_50settlement_start,t_50settlement_end,t_stepgrievance_ruling],ymin=0,ymax=1,transform=plt.gca().get_xaxis_transform(),color='k')
plt.annotate(xy=[t_grievance_submission,3400],text='Greivances over 50% and \nStep Lowering Filed',rotation='vertical',ha='right')
plt.annotate(xy=[t_2022contract_ratification,3200],text=' 2022 Contract Ratified',rotation='vertical',ha='right')
plt.annotate(xy=[t_50settlement_start,3200],text='50% Grievance Goes to Settlement',rotation='vertical',ha='right')
plt.annotate(xy=[t_50settlement_end,3400],text='50% Pay Starts',rotation='vertical',ha='right')
plt.annotate(xy=[t_stepgrievance_ruling,3100],text='Arbitration Rules UC Violated Contract by Step Lowering',rotation='vertical',ha='right')


plt.xlabel('Date')
plt.ylabel('Monthly Salary ($)')
plt.title('GSR Salaries Over Time')
plt.legend(loc='upper left',fontsize=14)
plt.grid(True,ls='--',alpha=0.5)
plt.tight_layout()
plt.show()


fig.savefig('salary_timeseries_since_2022_contract.png',dpi=300,transparent=True)


# Plot losses

fig, ax = plt.subplots(figsize=(9,6))
plt.suptitle('Your Lost Wages Due to UC Contract Violations',y=.97,x=0.55,fontweight='bold')
ax.grid(alpha=0.4)

postcontractstudentlosses=[np.nan,losses_2023, losses_2024]
postquallosses = [postqual_losses,np.nan,np.nan]
prequallosses = [prequal_losses,np.nan,np.nan]

titles = ['Pre-2022 Contract \n Cohorts','2023 Cohort','2024 Cohort']

x = np.arange(len(postcontractstudentlosses))
width = 0.5 
offset = 0.25

# 2023 and 2024 cohorts 
bar3 = ax.bar(x,-1*np.round(np.array(postcontractstudentlosses)),width,label='Losses',color='tab:red') 
ax.bar_label(bar3, label_type='edge',padding=2,fontsize=12)


# older students post qualifying 
bar1 = ax.bar(x-offset,-1*np.round(np.array(postquallosses)),width,color='indianred') 
ax.bar_label(bar1, label_type='edge',padding=2,fontsize=12)
ax.annotate(xy=[x[0]-offset,6000],text='Post-\nQualifying',fontsize=12,ha='center')


# older students pre qualifying 
bar2 = ax.bar(x+offset,-1*np.round(np.array(prequallosses)),width,color='firebrick') 
ax.bar_label(bar2, label_type='edge',padding=2,fontsize=12)
ax.annotate(xy=[x[0]+offset,6000],text='Pre-\n Qualifying',fontsize=12,ha='center')


ax.set_ylabel('Cumulative Wages Lost \n Per Student ($)')
ax.set_title('As of July 2025',fontsize=12)
ax.set_xticks(x , titles)
ax.legend()
plt.tight_layout()
plt.show()

fig.savefig('losses_per_sio_student_since_contract_july2025.png',dpi=300,transparent=True)



fig, ax = plt.subplots(figsize=(9,6))
plt.suptitle('Your Lost Wages Due to UC Contract Violations',y=.94,x=0.55,fontweight='bold')
ax.grid(alpha=0.4)

postcontractstudentlosses=[np.nan,losses_2023, losses_2024]
postquallosses = [postqual_losses,np.nan,np.nan]
prequallosses = [prequal_losses,np.nan,np.nan]

titles = ['Pre-2022 Contract \n Cohorts','2023 Cohort','2024 Cohort']

x = np.arange(len(postcontractstudentlosses))
width = 0.5 
offset = 0.25


thiscmap = cmocean.cm.thermal # remember, can do reverse with _r at end of cmap name 
cpos = np.linspace(0.1,0.85,4)
colors = [thiscmap(c) for c in cpos] 

# older students post qualifying 
bar1 = ax.bar(x-offset,-1*np.round(np.array(postquallosses)),width,color=colors[0]) 
ax.bar_label(bar1, label_type='edge',padding=2,fontsize=12)
ax.annotate(xy=[x[0]-offset,6000],text='Post-\nQualifying',fontsize=12,ha='center',color='white')

# older students pre qualifying 
bar2 = ax.bar(x+offset,-1*np.round(np.array(prequallosses)),width,color=colors[1]) 
ax.bar_label(bar2, label_type='edge',padding=2,fontsize=12)
ax.annotate(xy=[x[0]+offset,6000],text='Pre-\n Qualifying',fontsize=12,ha='center',color='white')

# 2023 and 2024 cohorts 
bar3 = ax.bar(x,-1*np.round(np.array(postcontractstudentlosses)),width,label='Losses',color=colors[1:4]) 
ax.bar_label(bar3, label_type='edge',padding=2,fontsize=12)

ax.set_ylabel('Cumulative Wages Lost \n Per Student ($)')
ax.set_title('As of July 2025',fontsize=12,x=.45)
ax.set_xticks(x , titles)
ax.legend()
ax.grid(True,alpha=0.6,ls=':')
plt.tight_layout()
plt.show()

fig.savefig('losses_per_sio_student_since_contract_july2025_colors.png',dpi=300,transparent=True)
